In [1]:
!pip install -U sqlalchemy
!pip install -U psycopg2-binary
!pip install -U requests

In [2]:
# these are some helper functions
import sierraAPI
import json
import requests
from sqlalchemy import create_engine, text
from datetime import datetime

# read the configuration file
try:
    with open("config.json", "r") as f:
        config = json.load(f)

    client_key = config["client_key"]
    client_secret = config["client_secret"]
    base_url = config["base_url"]
    db_connection_string = config["db_connection_string"]
    print(f"base_url: {base_url}")
    
except:
    # TODO: logging
    exit()
    
    
# connect to the Sierra DB
try:
    sierra_engine = create_engine(db_connection_string)
    # print(f"sierra_engine.url: {sierra_engine.url}")

except:
    # TODO: logging
    exit()
    


# get the header for API authorization
try:
    headers = sierraAPI.get_access_headers(
        client_key=client_key, 
        client_secret=client_secret, 
        base_url=base_url
    )
    
    r = requests.get(base_url + 'info/token', headers=headers, verify=True)
    print(f"token expires in: {r.json()['expiresIn']}")

except:
    # TODO: logging
    exit()

base_url: https://classic.cincinnatilibrary.org:443/iii/sierra-api/v6/
token expires in: 3599


In [3]:
# get relevant information related to patrons who have turned 18 on this date
sql = """\
select
    rm.record_num as patron_record_num,
    pr.ptype_code,
    pr.expiration_date_gmt::date as expiration_date,
    (
        select 
            v.field_content 
        from
            sierra_view.varfield as v
        where
            v.record_id = rm.id 
            and v.varfield_type_code = 'b'
        order by 
            v.occ_num 
        limit 
            1
    ) as barcode
FROM 
    sierra_view.record_metadata as rm 
    join sierra_view.patron_record as pr on (
        pr.record_id = rm.id 
        and pr.ptype_code in (
            0 , 1 , 2 , 5 , 6 , 7 , 30 , 31 , 32
        )
        and (pr.birth_date_gmt + interval '18 years')::date = now()::date 
    )
WHERE 
    rm.record_type_code = 'p'
    and rm.campus_code = ''
"""

In [4]:
# this is the message we'll patch to the patron record
data = {
    "varFields": [
    {
      "fieldTag": "m",
      "content": f"{datetime.now().strftime('%m/%d/%Y')} User turned 18.  Need agreement signed."
    }
  ]
}

In [5]:
# r = requests.get(base_url + 'info/token', headers=headers, verify=True)

In [ ]:
with sierra_engine.connect() as connection:
    try:
        result = connection.execute(text(sql))
    except:
        pass
        # TODO: logging
    for i, row in enumerate(result):
        print(i, row, end=', ')
        
        try:
            r = requests.put(
                url=f"{base_url}patrons/{row['patron_record_num']}",
                headers=headers,
                json=data
            )
            print(r.status_code)
        # break
        # print("username:", row["username"])
        
        except:
            pass
            # TODO logging

In [7]:
r.status_code

204